In [2]:
import pandas as pd
import numpy as np

In [14]:
cols = ['movie id', 'movie title', 'release date', 'video release date',
              'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
              'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
              'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/ml-100k/u.item', sep='|', names=cols, encoding='latin-1')
movies['movie title']

0                                        Toy Story (1995)
1                                        GoldenEye (1995)
2                                       Four Rooms (1995)
3                                       Get Shorty (1995)
4                                          Copycat (1995)
5       Shanghai Triad (Yao a yao yao dao waipo qiao) ...
6                                   Twelve Monkeys (1995)
7                                             Babe (1995)
8                                 Dead Man Walking (1995)
9                                      Richard III (1995)
10                                   Seven (Se7en) (1995)
11                             Usual Suspects, The (1995)
12                                Mighty Aphrodite (1995)
13                                     Postino, Il (1994)
14                              Mr. Holland's Opus (1995)
15                     French Twist (Gazon maudit) (1995)
16                             From Dusk Till Dawn (1996)
17            

In [15]:
themes = cols[5:]
genres = []
for i in range(len(movies)):
    g = []
    for j in range(len(themes)):
        theme = themes[j]
        if (int(movies.iat[i, j + 5]) == 1):
            g.append(theme)
    genres.append(g)

movies['genres'] = genres

movies.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genres
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,"[Animation, Children's, Comedy]"
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,"[Action, Adventure, Thriller]"
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,[Thriller]
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,"[Action, Comedy, Drama]"
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,"[Crime, Drama, Thriller]"


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
movies['genres'] = movies['genres'].fillna("").astype('str')
tfidf_matrix = tf.fit_transform(movies['genres'])
tfidf_matrix.shape

(1682, 114)

In [20]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(1682, 1682)

In [37]:
movies
titles = movies['movie title']
indices = pd.Series(movies.index, index = titles)


def recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    print(sim_scores[1:21])
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

recommendations('Toy Story (1995)')

[(1, 0.0), (2, 0.0), (3, 0.0673411680091013), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.5608599404898935), (8, 0.0), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.23778799180430915), (13, 0.0), (14, 0.0), (15, 0.09537448822994837), (16, 0.03758773721586041), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.03771260348870099)]


421                Aladdin and the King of Thieves (1996)
1218                                Goofy Movie, A (1995)
101                                Aristocats, The (1970)
403                                      Pinocchio (1940)
624                        Sword in the Stone, The (1963)
945                         Fox and the Hound, The (1981)
968           Winnie the Pooh and the Blustery Day (1968)
1065                                         Balto (1995)
1077                              Oliver & Company (1988)
1408                            Swan Princess, The (1994)
1411    Land Before Time III: The Time of the Great Gi...
1469                              Gumby: The Movie (1995)
94                                         Aladdin (1992)
62                               Santa Clause, The (1994)
93                                      Home Alone (1990)
137                           D3: The Mighty Ducks (1996)
138                                  Love Bug, The (1969)
224           